# SQL на Python

Создаем соединение с нашей базой данных (их может быть больше одного). Если файла в скобках нет, он создается сам. <br> Я взяла тестовую базу данных <a href="https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite">отсюда</a>.

In [ ]:
import sqlite3

conn = sqlite3.connect('Chinook_Sqlite.sqlite')

Создаем курсор - это специальный объект который делает запросы и получает их результаты

In [ ]:
cursor = conn.cursor()

### Добываем информацию из таблицы:
<b>SELECT column_name1, column_name2... FROM table</b> - получаем двумерный массив с инфой из нужных столбиков, а если нужна вся таблица, вместо названий столбиков можно написать <b>*</b>
<br>
<b>ORDER BY</b> - порядок
<br>
<b>LIMIT 3</b> - лимит количества выданных данных, в данном случае 3

In [ ]:
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3")

# Получаем результат сделанного запроса
results = cursor.fetchall()
results2 =  cursor.fetchall()

print(results)   # [('A Cor Do Som',), ('Aaron Copland & London Symphony Orchestra',), ('Aaron Goldberg',)]
print(results2)  # []

После получения результата из курсора, второй раз без повторения самого запроса егo получить нельзя — вернется пустой результат!

## Запись в базу данных

In [ ]:
# Делаем INSERT запрос к базе данных, используя обычный SQL-синтаксис
cursor.execute("insert into Artist values (Null, 'A Aagrh!') ")

# Если мы не просто читаем, но и вносим изменения в базу данных - необходимо сохранить транзакцию
conn.commit()

# Проверяем результат
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3")
results = cursor.fetchall()
print(results)  # [('A Aagrh!',), ('A Cor Do Som',), ('Aaron Copland & London Symphony Orchestra',)]

Без <b>NULL</b> будет sqlite3.OperationalError: table Artist has 2 columns but 1 values were supplied<br><br>Если к базе установлено несколько соединений и одно из них осуществляет модификацю базы, то база SQLite залочивается до завершения (метод соединения <b>.commit()</b>) или отмены (метод соединения <b>.rollback()</b>) транзакции.
## Разбиваем запрос на несколько строк в тройных кавычках

Длинные запросы можно разбивать на несколько строк в произвольном порядке, если они заключены в тройные кавычки — одинарные ('''…''') или двойные ("""...""")

In [ ]:
cursor.execute("""
  SELECT name
  FROM Artist
  ORDER BY Name LIMIT 3
""")

Конечно, в таком простом примере разбивка не имеет смысла, но на сложных длинных запросах она может кардинально повышать читаемость кода.

## Объединяем запросы к базе данных в один вызов метода

Метод курсора <b>.execute()</b> позволяет делать только один запрос за раз, при попытке сделать несколько через точку с запятой будет ошибка. Вот так вот ничего не выйдет:

In [ ]:
cursor.execute("""
    insert into Artist values (Null, 'A Aagrh!');
    insert into Artist values (Null, 'A Aagrh-2!');
""")
# sqlite3.Warning: You can only execute one statement at a time.

Для решения такой задачи можно либо несколько раз вызывать метод курсора .execute(), либо использовать метод курсора <b>.executescript()</b>

In [ ]:
cursor.executescript("""
 insert into Artist values (Null, 'A Aagrh!');
 insert into Artist values (Null, 'A Aagrh-2!');
""")

Данный метод также удобен, когда у нас запросы сохранены в отдельной переменной или даже в файле и нам его надо применить такой запрос к базе.

## Подстановка значений в запрос
<b>Важно!</b> Никогда, ни при каких условиях, не используйте конкатенацию строк (+) или интерполяцию параметра в строке (%) для передачи переменных в SQL запрос. Такое формирование запроса, при возможности попадания в него пользовательских данных – это ворота для SQL-инъекций!
<br>
Правильный способ – использование второго аргумента метода .execute()
<br>
Возможны два варианта, но ни одним из этих способов не получится заменять имена таблиц: 

In [ ]:
# C подставновкой по порядку на места знаков вопросов:
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT ?", ('2'))

# И с использованием именнованных замен:
cursor.execute("SELECT Name from Artist ORDER BY Name LIMIT :limit", {"limit": 3})

## И многое другое, что можно делать с базой данных

Делаем множественную вставку строк проходя по коллекции с помощью метода курсора <b>.executemany()</b>

In [ ]:
# Обратите внимание, даже передавая одно значение - его нужно передавать кортежем!
# Именно по этому тут используется запятая в скобках!
new_artists = [
    ('A Aagrh!',),
    ('A Aagrh!-2',),
    ('A Aagrh!-3',),
]
cursor.executemany("insert into Artist values (Null, ?);", new_artists)

Получаем результаты по одному, используя метод курсора <b>.fetchone()</b>
<br>
Он всегда возвращает кортеж или <b>None</b>. если запрос пустой.<br>
<u><b>Важно!</b></u> Стандартный курсор забирает все данные с сервера сразу, не зависимо от того, используем мы .fetchall() или .fetchone()

In [ ]:
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3")
print(cursor.fetchone())    # ('A Cor Do Som',)
print(cursor.fetchone())    # ('Aaron Copland & London Symphony Orchestra',)
print(cursor.fetchone())    # ('Aaron Goldberg',)
print(cursor.fetchone())    # None

Курсор как итератор:

In [ ]:
# Использование курсора как итератора
for row in cursor.execute('SELECT Name from Artist ORDER BY Name LIMIT 3'):
        print(row)
# ('A Cor Do Som',)
# ('Aaron Copland & London Symphony Orchestra',)
# ('Aaron Goldberg',)

## Повышаем устойчивость кода 

Для большей устойчивости программы (особенно при операциях записи) можно оборачивать инструкции обращения к БД в блоки «try-except-else» и использовать встроенный в sqlite3 «родной» объект ошибок, например, так:

In [ ]:
try:
    cursor.execute(sql_statement)
    result = cursor.fetchall()
except sqlite3.DatabaseError as err:       
    print("Error: ", err)
else:
    conn.commit()